In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import os
import glob

In [2]:
data_path='dataset'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels)) #empty dictionary

print(label_dict)
print(categories)
print(labels)

{'with mask': 0, 'without mask': 1}
['with mask', 'without mask']
[0, 1]


In [3]:
img_size=100
data=[]
target=[]


for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            #gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
            #Coverting the image into gray scale
            resized=cv2.resize(img,(img_size,img_size))
            #resizing the gray scale into 50x50, since we need a fixed common size for all the images in the dataset
            data.append(resized)
            target.append(label_dict[category])
            #appending the image and the label(categorized) into the list (dataset)

        except Exception as e:
            print('Exception:',e)
            #if any exception rasied, the exception will be printed here. And pass to the next image

In [4]:
data[0].shape

(100, 100, 3)

In [5]:
data=np.array(data)/255.0

In [6]:
data.shape

(1376, 100, 100, 3)

In [7]:
#data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)

from keras.utils import np_utils

new_target=np_utils.to_categorical(target)

Using TensorFlow backend.


In [8]:
new_target

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [9]:
target

array([0, 0, 0, ..., 1, 1, 1])

In [10]:
model=Sequential()
model.add(Conv2D(200,(3,3), activation='relu', input_shape=data.shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2)))
#First CNN Layer

model.add(Conv2D(100,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))
#Second CNN Layer

model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(2,activation='sigmoid'))
#The Final layer with two outputs for two categories

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [11]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(data,new_target,test_size=0.1)

In [12]:
model.fit(train_data,train_target, batch_size = 32, epochs = 30,validation_split=0.2)

Epoch 1/30
31/31 [==============================] - 103s 3s/step - loss: 0.5724 - accuracy: 0.7394 - val_loss: 0.2073 - val_accuracy: 0.9234
Epoch 2/30
31/31 [==============================] - 123s 4s/step - loss: 0.1766 - accuracy: 0.9394 - val_loss: 0.0986 - val_accuracy: 0.9637
Epoch 3/30
31/31 [==============================] - 124s 4s/step - loss: 0.0959 - accuracy: 0.9697 - val_loss: 0.0821 - val_accuracy: 0.9718
Epoch 4/30
31/31 [==============================] - 117s 4s/step - loss: 0.0863 - accuracy: 0.9687 - val_loss: 0.0729 - val_accuracy: 0.9798
Epoch 5/30
31/31 [==============================] - 193s 6s/step - loss: 0.0769 - accuracy: 0.9737 - val_loss: 0.0752 - val_accuracy: 0.9839
Epoch 6/30
31/31 [==============================] - 174s 6s/step - loss: 0.0555 - accuracy: 0.9798 - val_loss: 0.0793 - val_accuracy: 0.9839
Epoch 7/30
31/31 [==============================] - 171s 6s/step - loss: 0.0387 - accuracy: 0.9889 - val_loss: 0.0989 - val_accuracy: 0.9798
Epoch 8/30
31

In [13]:
pred=model.predict(test_data)

In [18]:
print(model.evaluate(test_data,test_target))

5/5 [==============================] - 2s 439ms/step - loss: 0.1966 - accuracy: 0.9783
[0.196571946144104, 0.97826087474823]


In [19]:
model.save('face_mask_detection.model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: face_mask_detection.model\assets
